In [1]:
# Task 3: Credit Risk Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
risk = pd.read_csv('Task 3 and 4_Loan_Data.csv')

In [3]:
risk.head(4)

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X = risk[['credit_lines_outstanding', 'loan_amt_outstanding',
        'total_debt_outstanding', 'income', 'years_employed', 'fico_score']]

y = risk['default']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)

In [10]:
 from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [13]:
scaler = StandardScaler()
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy Score:', accuracy)

Accuracy Score: 0.996


In [16]:
# Check the probability a borrower will default
pd_val = model.predict_proba(X_test)[:, 1]

In [17]:
print(pd_val)

[2.54803489e-02 8.78819679e-01 1.00000000e+00 ... 1.00000000e+00
 7.39953128e-23 9.54813180e-15]


In [18]:
# Expected loss
loan_amounts = X_test['loan_amt_outstanding']
LGD = 0.90

expected_loss = loan_amounts * pd_val * LGD
#print(expected_loss)

In [19]:
# Function to output expected loss
def calculate_expected_loss(model, features, loan_amount, recovery_rate=0.10):
    if isinstance(features, pd.Series):
      features = features.to_frame().T
    pd_val = model.predict_proba(features)[0][1]
    lgd = 1 - recovery_rate
    return loan_amount * pd_val * lgd



In [20]:
# Test the function
test_features = X_test.iloc[[0]]
test_loan_amt = test_features['loan_amt_outstanding'].values[0]

loss = calculate_expected_loss(model, test_features, test_loan_amt)
print("Expected Loss:", loss)

Expected Loss: 85.92342695478634


In [21]:
# Task 4: Bucket FICO Scores

In [22]:
from sklearn.preprocessing import KBinsDiscretizer

In [23]:
# lower bucket number = better credit score
n_buckets = 5

#KMeans binning for Fico Score
dsctz = KBinsDiscretizer(n_bins=n_buckets, encode='ordinal', strategy='kmeans')
risk['fico_bucket'] = dsctz.fit_transform(risk[['fico_score']]).astype(int)

# lower bucket number = better credit score
risk['fico_bucket'] = (n_buckets - 1) - risk['fico_bucket']

risk[['fico_score', 'fico_bucket']].head()

,fico_score,fico_bucket
0,605,3
1,572,3
2,602,3
3,612,2
4,631,2
